# 第十三章 Python建模库介绍

# 13.1 pandas与模型代码的接口
- 模型开发的通常工作流是使用pandas进行数据加载和清洗，然后切换到建模库进行建
模。开发模型的重要一环是机器学习中的“特征工程”。它可以描述从原始数据集中提
取信息的任何数据转换或分析，这些数据集可能在建模中有用。本次学习的数据聚合
和GroupBy工具常用于特征工程中
- pandas与其它分析库通常是靠NumPy的数组联系起来的。将DataFrame转换为NumPy
数组，可以使用.values属性

In [1]:
import pandas as pd
import numpy as np

data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [2]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [3]:
data.index

RangeIndex(start=0, stop=5, step=1)

In [4]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

- 要转换回DataFrame，可以传递一个二维ndarray，可带有列名

In [5]:
df2 = pd.DataFrame(data.values,columns=['one','two','three'],index=['a','b','c','d','e'])
df2

,one,two,three
a,1.0,0.01,-1.5
b,2.0,-0.01,0.0
c,3.0,0.25,3.6
d,4.0,-4.10,1.3
e,5.0,0.00,-2.0


In [6]:
df2.index.name = 'rows_name'
df2.columns.name = 'cols_name'
df2

cols_name,one,two,three
rows_name,,,
a,1.0,0.01,-1.5
b,2.0,-0.01,0.0
c,3.0,0.25,3.6
d,4.0,-4.10,1.3
e,5.0,0.00,-2.0


##### 笔记：最好当数据是均匀的时候使用.values属性。例如，全是数值类型。如果数据是不均匀的，结果会是Python对象的ndarray

In [8]:
df3 = data.copy()
df3['str'] = ['a','b','c','d','e']
df3

,x0,x1,y,str
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [10]:
df3.values  #貌似没啥区别？

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

- 只想使用列的子集。建议使用loc，用values作索引

In [12]:
# 是用columns的意思吗？
data.loc[:,['x0','x1']].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

- 添加category列并使用get_dummies转成虚变量

In [13]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                 categories = ['a','b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


替换category列为虚变量，创建虚变量，删除category列，然后添
加到结果：

In [14]:
dummies = pd.get_dummies(data.category,prefix='category')
dummies

,category_a,category_b
0,1,0
1,0,1
2,1,0
3,1,0
4,0,1


In [15]:
data_with_dummies = data.drop('category',axis=1).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


## 13.2 用Patsy创建模型描述
Patsy是Python的一个库，使用简短的字符串“公式语法”描述统计模型（尤其是线性模
型），可能是受到了R和S统计编程语言的公式语法的启发。Patsy适合描述statsmodels的线性模型，Patsy的公式是一个特殊的字符串语法，如下所示：

- a+b不是将a与b相加的意思，而是为模型创建的设计矩阵。
- patsy.dmatrices函数接收一个公式字符串和一个数据集（可以是DataFrame或数组的字典），为线性模型创建设计
矩阵

In [16]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [17]:
import patsy
y, X = patsy.dmatrices('y~x0+x1',data)
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [18]:
X
# 结果中的intercept是截距，是线性模型的惯例用法

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

- Intercept是哪里来的。这是线性模型（比如普通最小二乘回归）的惯例用法。
添加 +0 到模型可以不显示intercept：

In [19]:
patsy.dmatrices('y~x0+x1+0',data)

(DesignMatrix with shape (5, 1)
      y
   -1.5
    0.0
    3.6
    1.3
   -2.0
   Terms:
     'y' (column 0),
 DesignMatrix with shape (5, 2)
   x0     x1
    1   0.01
    2  -0.01
    3   0.25
    4  -4.10
    5   0.00
   Terms:
     'x0' (column 0)
     'x1' (column 1))

#### Patsy对象可以直接传递到算法（比如numpy.linalg.lstsq）中，它执行普通最小二乘回归：

In [20]:
coef,resid,_,_ = np.linalg.lstsq(X,y,)

d:\python35\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [21]:
coef,resid,_,_ = np.linalg.lstsq(X,y,rcond=None)

In [22]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

- 型的元数据保留在design_info属性中，因此你可以重新附加列名到拟合系数，以获得
一个Series，例如：

In [24]:
coef = pd.Series(coef.squeeze(),index=X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

In [25]:
X.design_info

DesignInfo(['Intercept', 'x0', 'x1'],
           factor_infos={EvalFactor('x0'): FactorInfo(factor=EvalFactor('x0'),
                                    type='numerical',
                                    state=<factor state>,
                                    num_columns=1),
                         EvalFactor('x1'): FactorInfo(factor=EvalFactor('x1'),
                                    type='numerical',
                                    state=<factor state>,
                                    num_columns=1)},
           term_codings=OrderedDict([(Term([]),
                                      [SubtermInfo(factors=(),
                                                   contrast_matrices={},
                                                   num_columns=1)]),
                                     (Term([EvalFactor('x0')]),
                                      [SubtermInfo(factors=(EvalFactor('x0'),),
                                                   contrast_matrices={},
     

### 用Patsy公式进行数据转换

- 以将Python代码与patsy公式结合。在评估公式时，库将尝试查找在封闭作用域内使
用的函数：

In [26]:
y,X=patsy.dmatrices('y~x0+np.log(np.abs(x1)+1)',data,)

In [28]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

- 常见的变量转换包括标准化（平均值为0，方差为1）和中心化（减去平均值）。Patsy有
内置的函数进行这样的工作：

In [30]:
y,X=patsy.dmatrices('y~standardize(x0) + center(x1)',data)

In [31]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

- 作为建模的一步，你可能拟合模型到一个数据集，然后用另一个数据集评估模型。另一
个数据集可能是剩余的部分或是新数据。
- 当执行中心化和标准化转变，用新数据进行预
测要格外小心。因为你必须使用平均值或标准差转换新数据集，这也称作状态转换。
- patsy.build_design_matrices函数可以使用原始样本数据集的保存信息，来转换新数
据，：

In [32]:
new_data = pd.DataFrame({
    'x0': [6, 7, 8, 9],
    'x1': [3.1, -0.5, 0, 2.3],
    'y': [1, 2, 3, 4]})

In [36]:
print(X.design_info)
print(X)

DesignInfo(['Intercept', 'standardize(x0)', 'center(x1)'],
           factor_infos={EvalFactor('center(x1)'): FactorInfo(factor=EvalFactor('center(x1)'),
                                    type='numerical',
                                    state=<factor state>,
                                    num_columns=1),
                         EvalFactor('standardize(x0)'): FactorInfo(factor=EvalFactor('standardize(x0)'),
                                    type='numerical',
                                    state=<factor state>,
                                    num_columns=1)},
           term_codings=OrderedDict([(Term([]),
                                      [SubtermInfo(factors=(),
                                                   contrast_matrices={},
                                                   num_columns=1)]),
                                     (Term([EvalFactor('standardize(x0)')]),
                                      [SubtermInfo(factors=(EvalFactor('standardiz

In [38]:
# 意思是X中的数据处理方法传递给了new_data,new_date执行并返回结果
new_X = patsy.build_design_matrices([X.design_info] ,new_data)
# z注意X.design_info在中括号里
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [39]:
y,X = patsy.dmatrices('y~ I(x0 + x1)', data)
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

### 分类数据和Patsy

- 当你在Patsy公式中使用非数值数据，它们会默认转换为虚变量。如果有截距，会去掉一
个，避免共线性：

In [40]:
data = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})
data

,key1,key2,v1,v2
0,a,0,1,-1.0
1,a,1,2,0.0
2,b,0,3,2.5
3,b,1,4,-0.5
4,a,0,5,4.0
5,b,1,6,-1.2
6,a,0,7,0.2
7,b,0,8,-1.7


In [42]:
y,X = patsy.dmatrices('v2 ~ key1',data)
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [44]:
data1 = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'c', 'a', 'b', 'c', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})
y1,X1 = patsy.dmatrices('v2 ~ key1',data1)
X1

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key1[T.c]
          1          0          0
          1          0          0
          1          1          0
          1          0          1
          1          0          0
          1          1          0
          1          0          1
          1          1          0
  Terms:
    'Intercept' (column 0)
    'key1' (columns 1:3)

- 如果你从模型中忽略截距，每个分类值的列都会包括在设计矩阵的模型中：

In [45]:
data1 = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'c', 'a', 'b', 'c', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})
y1,X1 = patsy.dmatrices('v2 ~ key1 + 0',data1)
X1

DesignMatrix with shape (8, 3)
  key1[a]  key1[b]  key1[c]
        1        0        0
        1        0        0
        0        1        0
        0        0        1
        1        0        0
        0        1        0
        0        0        1
        0        1        0
  Terms:
    'key1' (columns 0:3)

- 使用C函数，数值列可以截取为分类量

In [57]:
data2 = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'c', 'a', 'b', 'c', 'b'],
    'key2': [0, 1, 2, 1, 0, 2, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})
y, X = patsy.dmatrices('v2 ~ C(key2) + key2 +0',data2)
X

DesignMatrix with shape (8, 4)
  C(key2)[0]  C(key2)[1]  C(key2)[2]  key2
           1           0           0     0
           0           1           0     1
           0           0           1     2
           0           1           0     1
           1           0           0     0
           0           0           1     2
           1           0           0     0
           1           0           0     0
  Terms:
    'C(key2)' (columns 0:3)
    'key2' (column 3)

- 当在模型中使用多个分类名，事情就会变复杂，因为会包括key1:key2形式的相交部
分，它可以用在方差（ANOVA）模型分析中

In [60]:
data = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})
data['key2'] = data['key2'].map({0:'zero',1:'one'})
print(data)
y,X = patsy.dmatrices('v2 ~ key1 + key2',data)
X

  key1  key2  v1   v2
0    a  zero   1 -1.0
1    a   one   2  0.0
2    b  zero   3  2.5
3    b   one   4 -0.5
4    a  zero   5  4.0
5    b   one   6 -1.2
6    a  zero   7  0.2
7    b  zero   8 -1.7


DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [61]:
y,X = patsy.dmatrices('v2 ~ key1 + key2 + 0',data)
X

DesignMatrix with shape (8, 3)
  key1[a]  key1[b]  key2[T.zero]
        1        0             1
        1        0             0
        0        1             1
        0        1             0
        1        0             1
        0        1             0
        1        0             1
        0        1             1
  Terms:
    'key1' (columns 0:2)
    'key2' (column 2)

In [62]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)#key1:key2这是与的意思?
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

# 13.3 statsmodels介绍
statsmodels是Python进行拟合多种统计模型、进行统计试验和数据探索可视化的库。
Statsmodels包含许多经典的统计方法，但没有贝叶斯方法和机器学习模型。
statsmodels包含的模型有：
- 线性模型，广义线性模型和健壮线性模型
- 线性混合效应模型
- 方差（ANOVA）方法分析时间序列过程和状态空间模型
- 广义矩估计

## 估计线性模型

- statsmodels有多种线性回归模型，包括从基本（比如普通最小二乘）到复杂（比如迭代
加权最小二乘法）的。
- statsmodels的线性模型有两种不同的接口：基于数组和基于公式。它们可以通过API模
块引入：

In [63]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

- 为了展示它们的使用方法，从一些随机数据生成一个线性模型：

In [80]:
def dnorm(mean,variance,size=1):
    if isinstance(size,int):
        size = size,
    return mean + np.sqrt(variance)*np.random.randn(*size)

In [79]:
def dnorm1(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * np.random.randn(*size)
dnorm1(0,1,size=20)

array([-0.57602533,  0.67969341,  1.210302  , -1.23013642,  0.88810349,
        1.24622217,  0.17190694, -0.04044873, -1.28985539,  1.01412047,
        0.20576101,  0.22786141, -0.92466827,  0.07606064,  0.42558542,
        0.43188934,  0.60473521,  0.99334923, -0.69421691,  1.86115391])

In [81]:
# 为了代码效果重现，使用seed
np.random.seed(12345)
N = 100
X = np.c_[dnorm(0,0.4,size=N),
         dnorm(0, 0.6, size=N),          
         dnorm(0, 0.2, size=N)]
eps = dnorm(0,0.1,size=N)
beta = [0.1,0.3,0.5]
y = np.dot(X, beta) + eps

In [82]:
X[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [83]:
np.set_printoptions(precision=4, suppress=True)

In [84]:
y[:5]

array([ 0.4279, -0.6735, -0.0909, -0.4895, -0.1289])

- 线性模型通常要拟合一个截距。sm.add_constant函数可以添加一
个截距的列到现存的矩阵

In [85]:
X_model = sm.add_constant(X)
X_model[:5]

array([[ 1.    , -0.1295, -1.2128,  0.5042],
       [ 1.    ,  0.3029, -0.4357, -0.2542],
       [ 1.    , -0.3285, -0.0253,  0.1384],
       [ 1.    , -0.3515, -0.7196, -0.2582],
       [ 1.    ,  1.2433, -0.3738, -0.5226]])

- sm.OLS类可以拟合一个普通最小二乘回归：

In [86]:
models = sm.OLS(y,X)

- 这个模型的fit方法返回了一个回归结果对象，它包含估计的模型参数和其它内容：

In [87]:
results = models.fit()

In [88]:
results.params

array([0.1783, 0.223 , 0.501 ])

- 对结果使用summary方法可以打印模型的详细诊断结果：

In [89]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.413
Method:                 Least Squares   F-statistic:                     24.42
Date:                Wed, 13 Mar 2019   Prob (F-statistic):           7.44e-12
Time:                        15:33:46   Log-Likelihood:                -34.305
No. Observations:                 100   AIC:                             74.61
Df Residuals:                      97   BIC:                             82.42
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1783      0.053      3.364      0.0

- 这里的参数名为通用名x1, x2等等。假设所有的模型参数都在一个DataFrame中：

In [90]:
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])
data['y'] = y
data[:5]

,col0,col1,col2,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941


- 用statsmodels的公式API和Patsy的公式字符串：

In [91]:
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()
results.params

Intercept    0.033559
col0         0.176149
col1         0.224826
col2         0.514808
dtype: float64

In [94]:
results.tvalues

Intercept    0.952188
col0         3.319754
col1         4.850730
col2         6.303971
dtype: float64

- 观察下statsmodels是如何返回Series结果的，附带有DataFrame的列名。当使用公式和
pandas对象时，我们不需要使用add_constant。
给出一个样本外数据，你可以根据估计的模型参数计算预测值：

In [95]:
results.predict(data[:5])

0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

## 估计时间序列过程
目前看工作中交界较小，暂不研究，了解即可。

- Kaggle竞赛的经典数据集，关于泰坦尼克号乘客的生还率。

In [96]:
train = pd.read_csv('datasets/titanic/train.csv')
test = pd.read_csv('datasets/titanic/test.csv')
train[:4]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


- statsmodels和scikit-learn通常不能接收缺失数据，因此我们要查看列是否包含缺失值：

In [98]:
train.isnull().head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,False,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,True,False


In [99]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

- 在统计和机器学习的例子中，根据数据中的特征，一个典型的任务是预测乘客能否生
还。模型现在训练数据集中拟合，然后用样本外测试数据集评估。
- 用年龄作为预测值，但是它包含缺失值。缺失数据补全的方法有多种，用一
种简单方法，用训练数据集的中位数补全两个表的空值：

In [107]:
impute_value = train['Age'].median()
train['Age'] = train['Age'].fillna(impute_value)
test['Age'] = test['Age'].fillna(impute_value)

- 现在需要指定模型。增加了一个列IsFemale，作为“Sex”列的编码：

In [108]:
train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

- 然后，确定一些模型变量，并创建NumPy数组：

In [109]:
predictors = ['Pclass', 'IsFemale', 'Age']
X_train = train[predictors].values
X_test = test[predictors].values
y_train = train['Survived'].values
X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [110]:
y_train[:5]

array([0, 1, 1, 1, 0], dtype=int64)

- 不能保证这是一个好模型，但它的特征都符合。用scikit-learn的
LogisticRegression模型，创建一个模型实例：

In [111]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

- 用模型的fit方法，将它拟合到训练数据：

In [112]:
model.fit(X_train, y_train)

d:\python35\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [113]:
y_predict = model.predict(X_test)
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0], dtype=int64)